<a href="https://colab.research.google.com/github/thien2603/Vietnamese-Sentiment-PEFT/blob/main/Ch%C3%A0o_m%E1%BB%ABng_b%E1%BA%A1n_%C4%91%E1%BA%BFn_v%E1%BB%9Bi_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets torch evaluate
!pip install accelerate -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.5 MB/s eta 0:00:00


In [2]:
import transformers
print(transformers.__version__)

4.57.1
4.57.1


In [15]:
import numpy as np
from datasets import load_dataset, DatasetDict
import evaluate
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    pipeline
)

# ==================================================================
# PHẦN 1: TẢI VÀ CHUẨN BỊ DỮ LIỆU (GIỮ NGUYÊN)
# ==================================================================
print("--- Bước 1: Tải và chuẩn bị dữ liệu (TỪ FILE LOCAL) ---")

# 1. Tải dataset từ các file CSV bạn đã cung cấp
data_files = {
    'train': 'train.csv',
    'test': 'test.csv',
    'validation': 'val.csv'
}
dataset = load_dataset('csv', data_files=data_files)
print(f"Đã tải xong dữ liệu từ file local:\n{dataset}")

# 2. Chuyển nhãn chữ (POS, NEG, NEU) sang số (0, 1, 2)
label2id = {'NEG': 0, 'NEU': 1, 'POS': 2}
id2label = {0: 'NEG', 1: 'NEU', 2: 'POS'}

def map_labels_to_int(example):
    example['label'] = label2id.get(example['label'], -1)
    return example

print("Đang chuyển nhãn chữ (POS, NEG, NEU) sang số...")
dataset = dataset.map(map_labels_to_int, batched=False)
dataset = dataset.filter(lambda example: example['label'] != -1)
print(f"Đã chuyển nhãn sang số:\n{dataset}")

# 3. Tải Tokenizer của PhoBERT
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

# 4. Hàm tiền xử lý
def preprocess_function(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=128
    )

# 5. Áp dụng hàm tiền xử lý
print("Đang token hóa dữ liệu...")
tokenized_datasets = dataset.map(preprocess_function, batched=True, remove_columns=['text'])
print("Tải và xử lý dữ liệu xong!")


# ==================================================================
# PHẦN 2: TẢI MODEL NỀN VÀ CẤU HÌNH (SỬA LỖI wandb)
# ==================================================================
print("\n--- Bước 2: Tải model nền và cấu hình ---")

# 1. Tải model PhoBERT nền (Giữ nguyên)
model = AutoModelForSequenceClassification.from_pretrained(
    "vinai/phobert-base",
    num_labels=3,
    id2label=id2label,
    label2id=label2id
)

# 2. Định nghĩa các tham số huấn luyện (ĐÃ SỬA)
training_args = TrainingArguments(
    output_dir="./results_sentiment_3label",
    num_train_epochs=1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,

    # Dùng cho phiên bản transformers cũ
    do_eval=True,
    save_steps=200,
    logging_steps=200,

    # --- SỬA LỖI KHÔNG CẦN API (wandb) ---
    # Thêm dòng này để tắt tính năng log lên wandb
    report_to="none",
    # --- KẾT THÚC SỬA LỖI ---

    fp16=False,
)

# 3. Định nghĩa hàm tính độ đo (Giữ nguyên)
metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(
        predictions=predictions,
        references=labels,
        average="weighted"
    )

# 4. Khởi tạo Trainer (Giữ nguyên)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"], # Dùng tập "validation"
    compute_metrics=compute_metrics,
)

print("Cấu hình huấn luyện xong!")


# ... (trước đó là trainer.train() như hiện tại của bạn)

# ==================================================================
# PHẦN 3 (sau khi train): đảm bảo lưu model + tokenizer
# ==================================================================
print("Đang huấn luyện...")
trainer.train()
print("Huấn luyện xong — lưu model và tokenizer vào thư mục kết quả...")

model_path = "./results_sentiment_3label"
trainer.save_model(model_path)            # lưu model + config
tokenizer.save_pretrained(model_path)     # lưu tokenizer
print(f"Saved model & tokenizer to {model_path}")

# Đánh giá model tốt nhất trên tập Test
print("\nĐánh giá model trên tập Test...")
eval_results = trainer.evaluate(eval_dataset=tokenized_datasets['test'])
print(f"Kết quả đánh giá trên tập Test: {eval_results}")

# ==================================================================
# PHẦN 4: Thử nghiệm model (dùng pipeline an toàn)
# ==================================================================
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline

# Cách an toàn: load model + tokenizer từ thư mục đã lưu
model_loaded = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer_loaded = AutoTokenizer.from_pretrained(model_path)

sentiment_pipeline = pipeline(
    "text-classification",
    model=model_loaded,
    tokenizer=tokenizer_loaded
)

# Thử nghiệm
text_positive = "Phim này hay thật sự, nội dung sâu sắc."
text_negative = "Shop làm ăn chán quá, giao hàng thì lâu, sản phẩm thì vỡ."
text_neutral = "Điện thoại này sẽ ra mắt vào tháng 12 tới."

print("Kết quả thử nghiệm:")
print(text_positive, "->", sentiment_pipeline(text_positive))
print(text_negative, "->", sentiment_pipeline(text_negative))
print(text_neutral, "->", sentiment_pipeline(text_neutral))


--- Bước 1: Tải và chuẩn bị dữ liệu (TỪ FILE LOCAL) ---
Đã tải xong dữ liệu từ file local:
DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 4080
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1050
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1020
    })
})
Đang chuyển nhãn chữ (POS, NEG, NEU) sang số...


Map:   0%|          | 0/4080 [00:00<?, ? examples/s]

Map:   0%|          | 0/1050 [00:00<?, ? examples/s]

Map:   0%|          | 0/1020 [00:00<?, ? examples/s]

Filter:   0%|          | 0/4080 [00:00<?, ? examples/s]

...

Step,Training Loss
200,0.889600


Huấn luyện xong — lưu model và tokenizer vào thư mục kết quả...
Saved model & tokenizer to ./results_sentiment_3label

Đánh giá model trên tập Test...


Kết quả đánh giá trên tập Test: {'eval_loss': 0.6865805387496948, 'eval_f1': 0.7058850022363163, 'eval_runtime': 2.2442, 'eval_samples_per_second': 467.867, 'eval_steps_per_second': 29.409, 'epoch': 1.0}


Device set to use cuda:0


Kết quả thử nghiệm:
Phim này hay thật sự, nội dung sâu sắc. -> [{'label': 'POS', 'score': 0.9343552589416504}]
Shop làm ăn chán quá, giao hàng thì lâu, sản phẩm thì vỡ. -> [{'label': 'NEG', 'score': 0.8816109895706177}]
Điện thoại này sẽ ra mắt vào tháng 12 tới. -> [{'label': 'NEU', 'score': 0.5094176530838013}]
